<div style="background-color: #f9f9f9; padding: 20px; border-left: 5px solid #2c3e50; border-radius: 5px; font-family: sans-serif;">

<h1 style="color: #2c3e50; margin-top: 0;">Image Encryption Project for CENG376 Image Processing</h1>

<p style="font-size: 1.1em; color: #555;">
This project implements a custom image encryption algorithm combining
<strong>Bitwise XOR operations</strong> with <strong>Matrix Permutation</strong>.
</p>

<h3 style="color: #e74c3c;">Key Features</h3>

<ul>
<li><strong>Dual-Layer Security:</strong> XOR + Permutation</li>
<li><strong>Image Reduction:</strong> 4/8 Neighbor averaging</li>
<li><strong>Restoration:</strong> Intelligent upscaling</li>
</ul>
</div>

<div style="background-color:#f9f9f9; padding:20px; border-left:5px solid #2c3e50; border-radius:6px; font-family:sans-serif;">

<h2 style="color:#2c3e50; margin-top:0;">What Is a Bitwise XOR Operation?</h2>

<p style="color:#555; font-size:1.05em;">
The <strong>bitwise XOR (exclusive OR)</strong> operation compares two numbers at the
<strong>binary level</strong>. Each pair of corresponding bits is processed independently
to produce a new binary result.
</p>

<h3 style="color:#2980b9;">How XOR Works</h3>

<p style="color:#555;">
XOR takes <strong>two binary digits</strong> and returns a single binary digit according to
the following rule:
</p>

<table style="border-collapse:collapse; margin:10px 0;">
<tr>
  <th style="border:1px solid #ccc; padding:6px;">Bit A</th>
  <th style="border:1px solid #ccc; padding:6px;">Bit B</th>
  <th style="border:1px solid #ccc; padding:6px;">A ⊕ B</th>
</tr>
<tr>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
</tr>
<tr>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
</tr>
<tr>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
</tr>
<tr>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">1</td>
  <td style="border:1px solid #ccc; padding:6px; text-align:center;">0</td>
</tr>
</table>

<h3 style="color:#2980b9;">Pixel Example</h3>

<p style="color:#555;">
Consider two grayscale pixel values:
</p>

<ul style="color:#555;">
  <li><strong>Pixel A:</strong> 150 → <code>10010110</code></li>
  <li><strong>Pixel B:</strong> 200 → <code>11001000</code></li>
</ul>

<p style="color:#555;">
Applying XOR bit by bit:
</p>

<pre style="background:#ecf0f1; padding:10px; border-radius:4px;">
10010110
⊕ 11001000
---------
01011110  → 94
</pre>

<h3 style="color:#2980b9;">Why Use XOR in Encryption?</h3>

<p style="color:#555;">
XOR is widely used in encryption because it is <strong>reversible</strong>. Applying the same
XOR operation twice with the same key restores the original value:
</p>

<pre style="background:#ecf0f1; padding:10px; border-radius:4px;">
( Pixel ⊕ Key ) ⊕ Key = Pixel
</pre>

<pre style="background:#ecf0f1; padding:10px; border-radius:4px;">
    That is 
    
Enrypted Image ⊕ Key = Decrypted Image
</pre>

<p style="color:#555;">
This property makes XOR ideal for image encryption, where pixel values can be
scrambled during encryption and perfectly recovered during decryption.
</p>

</div>


In [1]:
import numpy as np 
import cv2

In [2]:
class ImageCipher:
    def __init__(self , image , permutate=False):
        self.image = image
        self.height = image.shape[0]
        self.width = image.shape[1]
        self.permutation = permutate
        
        if(len(image.shape) == 3):
            self.color_channels = image.shape[2]
        else:
            self.color_channels = 1

        if self.permutation:
            total_pixels = self.height * self.width
            self.perm_indices = np.random.permutation(total_pixels)
        else:
            self.perm_indices = None
            
        self.key = self.generate_key(self.height,self.width , self.color_channels)

    def generate_key(self , h,w,c):
        key = np.random.randint(0 , 255 , size=(h ,w , c))
        key = key.astype('int')
        return key

    def encrypt_image(self):
        encrypted_image = np.bitwise_xor(self.image , self.key)

        if self.permutation:
            flat_image = encrypted_image.reshape(-1, self.color_channels)
            flat_image = flat_image[self.perm_indices]
            encrypted_image = flat_image.reshape(self.height, self.width, self.color_channels)
        
        return encrypted_image

    def decrypt_image(self , encrypted_image):
        if self.permutation :
            flat_shuffled = encrypted_image.reshape(-1 , self.color_channels)
            flat_ordered = np.zeros_like(flat_shuffled)
            flat_ordered[self.perm_indices] = flat_shuffled

            encrypted_image = flat_ordered.reshape(self.height , self.width , self.color_channels)
        
        decrypted_image = np.bitwise_xor(encrypted_image , self.key)
        return decrypted_image

## Matrix Permutation (Index-Based Shuffling)

Matrix permutation is an **index permutation**, meaning it rearranges **pixel positions**.

### How It Works
1. Flatten the image matrix into a 1D array.
2. Generate a random permutation of indices `0 … N−1`.
3. Reorder pixels using this index array.
4. Reshape back to the original matrix shape.



Matrix permutation here is an **index permutation**, meaning elements are reordered by indices, not modified. For example, let `x = [1, 2, 0]` be a permutation index array and `y = [4, 5, 6]` be the original data; applying the permutation gives `y[x] = [y[1], y[2], y[0]] = [5, 6, 4]`, which is a reordered version of `y`. In the image algorithm, the image matrix is first flattened into a 1D array of pixels, then a random permutation array is applied to shuffle pixel positions in the same way. To reverse this operation, an empty array is created and the shuffled values are placed back using the same indices: `original[x] = shuffled`. Because the permutation is one-to-one, every element returns to its exact original position, making the process perfectly reversible.


In [3]:
class ReducedCipher(ImageCipher):

    def __init__(self , image , method='4N'):
        super().__init__(image)
        self.method=method
        self.new_image , self.new_key = self.reduce()
    
    def reduce(self):
        reduced = []

        if self.method == "8N":
            step =3
        else : 
            step = 2
    
        for i in range(0, self.height - step + 1, step):
            row = []
            for j in range(0, self.width -step+ 1, step):

                if self.method == '4N':
                
                    p1 = self.image[i][j]
                    p2 = self.image[i+1][j]
                    p3 = self.image[i][j+1]
                    p4 = self.image[i+1][j+1]
                
                    avg = (p1.astype(float) + p2 + p3 + p4) / 4

                elif self.method == '8N':
                    total = 0.0

                    for r in range(3):
                        for c in range(3):
                            total += self.image[i+r][j+c]

                    avg = total/9
                   
                row.append(avg.astype(np.uint8))
            reduced.append(row)
        
        tempImg = np.array(reduced, dtype=np.uint8)
        new_key = self.generate_key(tempImg.shape[0], tempImg.shape[1], self.color_channels)
        return tempImg, new_key
    
    def encrypt_reduced(self):
        encrypted_reduced_image = np.bitwise_xor(self.new_image , self.new_key)
        return encrypted_reduced_image

    def decrypt_upgrade(self, encrypted_reduced_image):
        
        decrypted_reduced = np.bitwise_xor(encrypted_reduced_image, self.new_key)
        
        h_enc, w_enc = encrypted_reduced_image.shape[:2]
        restored = np.zeros((self.height, self.width, self.color_channels), dtype=np.uint8)

        if self.method == '8N':
            step = 3
        else:
            step=2
        
        for i in range(self.height):
            for j in range(self.width):
                r_idx = min(i // step, h_enc - 1)
                c_idx = min(j // step, w_enc - 1)
                
                restored[i, j] = decrypted_reduced[r_idx , c_idx]
        return restored

## Neighbor Averaging Reduction and Upscaling

The reduction process compresses the image by **averaging local pixel neighborhoods**. In **4-Neighbor (4N)** mode, each non-overlapping `2×2` block is replaced by the average of its four pixels, while in **8-Neighbor (8N)** mode, each `3×3` block is replaced by the average of its nine pixels. This produces a smaller image that preserves local intensity trends while reducing spatial resolution.

During upscaling, the reduced image is expanded back to the original size by **block replication**. Each pixel in the reduced image is copied into all positions of its corresponding `2×2` or `3×3` block using integer index mapping (`i // step`, `j // step`). This restores the image dimensions deterministically, without interpolation, ensuring consistency with the reduction step.


## A Possible Problem Application: End-to-End Image Encryption with Downscaling to Save Storage

<p align="center">
  <img src="diagram.png" alt="End-to-End Image Encryption Diagram">
</p>

In [8]:
img = cv2.imread('image.jpg')

In [9]:
imgClass = ImageCipher(img , permutate=True)

In [10]:
encrpytedImg = imgClass.encrypt_image()

In [11]:
cv2.imshow("encrpyed image" ,encrpytedImg.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
decrpyted = imgClass.decrypt_image(encrpytedImg)

In [13]:
cv2.imshow("decrpyted image" ,decrpyted.astype(np.uint8) )
cv2.waitKey(0)
cv2.destroyAllWindows()

In [14]:
reducedImage = ReducedCipher(img , method='8N')

In [15]:
encryptedImg = reducedImage.encrypt_reduced()

In [16]:
cv2.imshow("encrpyed image" ,encryptedImg.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [99]:
decryptedReducedImage = reducedImage.decrypt_upgrade(encryptedImg)

In [100]:
cv2.imshow("decrypted image" ,decryptedReducedImage.astype(np.uint8))
cv2.waitKey(0)
cv2.destroyAllWindows()